In [17]:
import polars as pl
import gc
from population_data import *

Aggregating by poste...

In [18]:
# List of indicators
indicators = ['FLT_REM_MNT', 'FLT_PAI_MNT']
x_columns = ['PRS_NAT']
relevant_columns = indicators + x_columns

dates = {2024 : ['0' + str(i) if i<10 else str(i) for i in range(1, 13)]}

data = pl.DataFrame()
df_poste = pl.DataFrame()

dataset_path = '/raid/datasets/allianzsante'

for year in dates.keys():
    for month in dates[year]:
        suffix = '.csv'

        # Current month    
        data = pl.read_csv(f'{dataset_path}/A{year}/A{year}{month}.csv', separator = ';', infer_schema_length=1000, columns = relevant_columns)
        
        # Computing RAC
        data = data.with_columns((pl.col('FLT_PAI_MNT') - pl.col('FLT_REM_MNT')).alias('RAC'))

        # Aggregating by PRS_NAT

        keys_to_group_by = ['PRS_NAT']
        all_indicators = indicators + ['RAC']
        all_keys = all_indicators + keys_to_group_by
        df_aggr = data[all_keys].group_by(keys_to_group_by).agg([pl.col(*all_indicators).sum()])

        df_poste = df_poste.vstack(df_aggr.group_by(['PRS_NAT']).agg(pl.col(*all_indicators).sum()))

        # Aggregating by category
        
        del data
        gc.collect()

# Aggregating by PRS_NAT last time and changing the code of PRS_NAT by its name

df_poste = df_poste[all_keys].group_by(keys_to_group_by).agg(pl.col(*all_indicators).sum())

print(df_poste)

shape: (1_038, 4)
┌─────────┬─────────────┬─────────────┬───────────┐
│ PRS_NAT ┆ FLT_REM_MNT ┆ FLT_PAI_MNT ┆ RAC       │
│ ---     ┆ ---         ┆ ---         ┆ ---       │
│ i64     ┆ f64         ┆ f64         ┆ f64       │
╞═════════╪═════════════╪═════════════╪═══════════╡
│ 2230    ┆ 33323.03    ┆ 3.9968e8    ┆ 3.9964e8  │
│ 1706    ┆ 157.0       ┆ 157.0       ┆ 0.0       │
│ 4326    ┆ 3.6097e6    ┆ 3.6097e6    ┆ 0.0       │
│ 1051    ┆ 356968.59   ┆ 357654.72   ┆ 686.13    │
│ 1182    ┆ 8.6242e7    ┆ 8.6242e7    ┆ 0.0       │
│ …       ┆ …           ┆ …           ┆ …         │
│ 1048    ┆ 4.9568e7    ┆ 4.9690e7    ┆ 121653.33 │
│ 3537    ┆ 84.72       ┆ 347.5       ┆ 262.78    │
│ 9435    ┆ 23115.0     ┆ 23115.0     ┆ 0.0       │
│ 4323    ┆ 3.7791e7    ┆ 3.7836e7    ┆ 45000.2   │
│ 3406    ┆ 4.7054e6    ┆ 4.7054e6    ┆ 0.0       │
└─────────┴─────────────┴─────────────┴───────────┘


Manipulating dictionaries to be able to map 'PRS_NAT' to a poste de soins

In [19]:
with open("dict_mapping.txt", "r", encoding="utf-8") as f:
    data_from_file = f.read()

dict_ID_to_PRS_NAT = eval(data_from_file)

dict_poste_to_id = {
    'Hospitalisation' : {"12", "13", "14", "15", "16", "17", "37"},
    'Dentaire' : {"18", "19", "20", "21", "22", "23"},
    'Optique' : {"24", "25", "26", "27", "28"},
    'Auditif' : {"29", "30", "31"},
    'Autre' : {"0", "32", "33", "34", "35", "36", "4", "5", "6", "7"},
    'Pharmacie': {"8", "9", "10", "11"},
    'Specialistes': {"1", "2", "3"}
}

dict_PRS_NAT_to_ID = {id: id_ for id_, prs_list in dict_ID_to_PRS_NAT.items() for id in prs_list}

dict_postes_to_PRS_NAT = {id: id_ for id_, prs_list in dict_poste_to_id.items() for id in prs_list}

df_poste_bbl = df_poste.with_columns(pl.col(*keys_to_group_by).cast(pl.String))
df_poste_bbl = df_poste_bbl.with_columns(pl.col(*keys_to_group_by).replace_strict(dict_PRS_NAT_to_ID, default=0))
df_poste_bbl = df_poste_bbl.with_columns(pl.col(*keys_to_group_by).replace_strict(dict_postes_to_PRS_NAT))
df_poste_bbl = df_poste_bbl[all_keys].group_by(keys_to_group_by).agg(pl.col(*all_indicators).sum())
df_poste_bbl.write_csv(f'bbl.csv', separator=';')
print(df_poste_bbl)

shape: (7, 4)
┌─────────────────┬─────────────┬─────────────┬────────────┐
│ PRS_NAT         ┆ FLT_REM_MNT ┆ FLT_PAI_MNT ┆ RAC        │
│ ---             ┆ ---         ┆ ---         ┆ ---        │
│ str             ┆ f64         ┆ f64         ┆ f64        │
╞═════════════════╪═════════════╪═════════════╪════════════╡
│ Pharmacie       ┆ 4.3447e10   ┆ 5.0449e10   ┆ 7.0024e9   │
│ Hospitalisation ┆ 1.5874e10   ┆ 2.0261e10   ┆ 4.3869e9   │
│ Dentaire        ┆ 3.7027e9    ┆ 1.3095e10   ┆ 9.3927e9   │
│ Optique         ┆ 6.0210e6    ┆ 3.3897e7    ┆ 2.7876e7   │
│ Specialistes    ┆ 7.7523e9    ┆ 9.6036e9    ┆ 1.8513e9   │
│ Autre           ┆ 8.6134e10   ┆ 7.2404e10   ┆ -1.3730e10 │
│ Auditif         ┆ 3.9411e8    ┆ 1.9385e9    ┆ 1.5444e9   │
└─────────────────┴─────────────┴─────────────┴────────────┘


In [20]:
dict_IA1 = {
    "Pharmacie": {1052},
    "Dentaire": {
        1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1412, 1413, 1414, 
        1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1431, 
        1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1451, 1452, 1453, 1454, 1455, 1456, 
        1457, 1459, 1460, 1461, 1462, 1463, 1464, 1465, 1466, 1467, 1470, 1471, 1472, 1473, 
        1474, 1475, 1476, 1477
    },
    "Optique": {1159},
    "Spécialistes": {
        1046, 1047, 1057, 1068, 1073, 1074, 1075, 1099, 1101, 1102, 1103, 1114, 1117, 1118, 
        1120, 1126, 1127, 1179, 1181, 1209, 1215, 1216
    },
    "Hospitalisation": {
        1121, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1319, 1320, 1321, 1322, 1323, 1324, 
        1325, 1331, 1332, 1333, 1335, 1336, 1341, 1342, 1351, 1352, 1353, 1354, 1361, 1511, 
        1512
    },
    "Auditif": set(),
    "Autres": {
        0, 1045, 1048, 1049, 1050, 1051, 1053, 1054, 1055, 1056, 1058, 1059, 1060, 1061, 1062, 
        1063, 1064, 1065, 1066, 1067, 1069, 1070, 1071, 1072, 1076, 1077, 1078, 1079, 1080, 
        1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 
        1095, 1096, 1097, 1098, 1100, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 
        1113, 1115, 1116, 1119, 1122, 1123, 1124, 1125, 1128, 1129, 1130, 1131, 1132, 1133, 
        1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 
        1148, 1149, 1150, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1160, 1161, 1162, 1163, 
        1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 
        1178, 1180, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 
        1194, 1195, 1196, 1197, 1198, 1199, 1201, 1202, 1203, 1210, 1211, 1212, 1213, 1214, 
        1218, 1221, 1222, 1224, 1225, 1226, 1227, 1228, 1521, 1522, 1523, 1601, 1602, 1603, 
        1604, 1605, 1606, 1607, 1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 
        1618, 1619, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631
    }
}

dict_IA2 = {
    "Pharmacie": {1961, 1966},
    "Dentaire": {1704, 1705, 1706},
    "Optique": set(),
    "Spécialistes": {
        1922, 1923, 1945
    },
    "Hospitalisation": {
        2100, 2101, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 
        2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 
        2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 
        2150, 2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158, 2159, 2160, 2161, 2162, 2163, 
        2164, 2165, 2166, 2167, 2168, 2169, 2170, 2171, 2172, 2173, 2174, 2175, 2176, 2177, 
        2178, 2179, 2180, 2181, 2182, 2183, 2184, 2185, 2186, 2187, 2188, 2189, 2190, 2191, 
        2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2204, 2205, 
        2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215, 2216, 2217, 2218, 2219, 
        2220, 2221
    },
    "Auditif": set(),
    "Autres": {
        1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 
        1646, 1647, 1648, 1649, 1650, 1651, 1652, 1653, 1654, 1701, 1702, 1703, 1707, 1708, 
        1711, 1712, 1715, 1716, 1717, 1718, 1721, 1722, 1723, 1724, 1725, 1726, 1727, 1728, 
        1729, 1731, 1732, 1733, 1734, 1735, 1736, 1741, 1742, 1743, 1744, 1745, 1746, 1747, 
        1748, 1749, 1751, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1771, 1781, 
        1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1791, 1811, 1812, 1813, 1814, 1821, 
        1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1901, 1903, 1904, 1905, 1906, 
        1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 
        1921, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 
        1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1946, 1947, 1948, 1949, 1951, 1952, 
        1954, 1955, 1956, 1957, 1959, 1960, 1962, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 
        1978, 1981, 1982, 1983, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999
    }
}

dict_IA3 = {
    "Pharmacie": {
        2231, 3311, 3312, 3313, 3314, 3315, 3316, 3317, 3318, 3319, 3320, 3321, 3322, 3323, 
        3324, 3325, 3326, 3327, 3328, 3329, 3330, 3331, 3332, 3333, 3334, 3335, 3336, 3341, 
        3342, 3343, 3344, 3345, 3346, 3347, 3348, 3349, 3350, 3351, 3352, 3353, 3354, 3355, 
        3356, 3357, 3358, 3359, 3360, 3361, 3362, 3363, 3364, 3365, 3366
    },
    "Dentaire": set(),
    "Optique": set(),
    "Spécialistes": {
        2411, 2412, 2413, 2414, 2415, 2416, 2417, 2418, 2419, 2420, 2421, 2422, 2423, 2424, 
        2425, 2426, 2428
    },
    "Hospitalisation": {
        2222, 2228, 2229, 2230, 2232, 2234, 2235, 2236, 2238, 2239, 2240, 2241, 2242, 2243, 
        2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 2252, 2253, 2254, 2255, 2256, 2257, 
        2258, 2259, 2260, 2261, 2262, 2263, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2271, 
        2272, 2273, 2274, 2275, 2276, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284, 2285, 
        2286, 2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295, 2297, 2298, 2299, 2300, 
        2301, 2302, 2303, 2304, 2309, 2321, 2331, 2332, 2333, 2334, 2335, 2336, 2337, 2338, 
        2339, 2341, 2342, 2343, 2344, 2345, 2346, 2347, 2351, 2352, 2353, 2354, 2355, 2356, 
        2357, 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, 2367, 2368, 2369, 2371, 
        2372, 2373, 2380, 2381, 2382, 2383, 2384, 2385, 2386, 2387, 2388, 2389, 2391, 2392
    },
    "Auditif": set(),
    "Autres": {
        2227, 2233, 2237, 2253, 2309, 2501, 2502, 2503, 2504, 3101, 3102, 3103, 3104, 3105, 
        3106, 3107, 3108, 3109, 3110, 3111, 3112, 3113, 3115, 3116, 3117, 3118, 3119, 3120, 
        3121, 3122, 3123, 3124, 3125, 3126, 3127, 3128, 3129, 3130, 3131, 3132, 3133, 3134, 
        3135, 3136, 3137, 3138, 3139, 3140, 3141, 3142, 3143, 3144, 3145, 3146, 3147, 3148, 
        3149, 3150, 3151, 3152, 3153, 3154, 3155, 3156, 3157, 3158, 3159, 3160, 3161, 3162, 
        3163, 3164, 3165, 3166, 3167, 3168, 3169, 3170, 3171, 3172, 3173, 3210, 3211, 3213, 
        3216, 3217, 3221, 3222, 3223, 3225, 3300, 3301, 3302, 3303, 3304, 3305, 3306, 3307, 
        3308, 3309, 3310
    }
}

dict_IA4 = {
    "Pharmacie": {
        3367, 3368, 3369, 3370, 3371, 3372, 3373, 3374, 3375, 3376, 3377, 3379, 3380, 3381,
        3382, 3383, 3384, 3385, 3386, 3387, 3388, 3389, 3390, 3391, 3392, 3393, 3394, 3395,
        3396, 3397, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 3407, 3408, 3410,
        3411, 3412, 3413, 3414, 3415, 3416, 3417, 3418, 3419, 3420, 3421, 3423, 3424, 3425
    },
    
    "Optique": {
        3500, 3501, 3502, 3503, 3504, 3505, 3506, 3507, 3508, 3509, 3510, 3519, 3520, 3523,
        3524, 3525, 3526, 3527, 3528, 3529, 3530, 3531, 3532, 3533, 3534, 3535, 3536, 3537,
        3538, 3539, 3553, 3554, 3555, 3556, 3557, 3559, 3560, 3562, 3563, 3564, 3565, 3566,
        3567, 3568, 3569, 3570, 3578, 3579, 3580, 3581, 3582, 3583, 3584, 3585, 3602
    },
    
    "Auditif": {3540, 3541, 3547, 3549, 3550, 3586, 3587, 3588, 3589, 3590, 3595},
    
    "Autres": {
        3511, 3512, 3513, 3514, 3515, 3516, 3517, 3518, 3521, 3522, 3542, 3543, 3544, 3545,
        3548, 3551, 3552, 3558, 3561, 3572, 3573, 3574, 3575, 3576, 3577, 3591, 3592, 3593,
        3594, 3596, 3597, 3598, 3599, 3600, 3601, 3610, 3611, 3612, 4111, 4112, 4113, 4114,
        4131, 4132, 4141, 4142, 4143, 4144, 4145, 4151, 4152, 4153, 4154, 4155, 4156, 4157,
        4158, 4159, 4203, 4204, 4205, 4206, 4207, 4208, 4209, 4210, 4211, 4212, 4213, 4214,
        4215, 4216, 4217, 4218, 4219, 4220, 4221, 4222, 4223, 4224, 4225, 4226, 4227, 4228,
        4229, 4230, 4231, 4232, 4233, 4234, 4235, 4236, 4237, 4238, 4239, 4240, 4241, 4310,
        4311, 4312, 4313, 4314, 4315, 4316, 4317, 4318, 4319, 4320, 4321, 4322, 4323, 4324,
        4325, 4326, 4327, 4328, 4329, 4330, 4331, 4332, 4333, 4339, 4341, 4342, 4343, 4351,
        4352, 4353, 4359, 4360, 4361, 4363, 4364, 4365, 4366, 4367, 4368, 4369, 4370, 4371,
        4372, 4373, 4374, 4375, 4376, 4377, 4378, 4379, 4380, 4381, 4382, 4391, 4392, 4393,
        4394, 4395, 4396, 4397, 4411, 4412, 4413, 4414, 4415, 4416, 4417, 4419, 4501, 4511,
        4512, 4513, 4514, 4515, 4611, 4612
    },
    
    "Dentaire": set(),
    "Spécialistes": set(),
    "Hospitalisation": set(),
}

dict_IA5 = {
    "Pharmacie": {9762, 9777},

    "Dentaire": {5201, 5202, 5203, 5204, 5205, 5206, 9702, 9703, 9704, 9705, 9716, 9761},

    "Optique": {
        5101, 5102, 5103, 5104, 5105, 5106, 5107, 5108, 5109, 5110, 5111, 5112, 5113, 5114,
        5115, 5116, 5117, 5118, 5119, 5120, 5121, 5122, 5123, 5124, 5125, 5126, 5127, 5128,
        5129, 5130, 9706, 9707, 9708, 9709, 9710, 9715, 9726
    },

    "Auditif": {5401, 5402, 5403, 9701, 9717, 9727, 9742, 9771},

    "Spécialistes": set(),

    "Hospitalisation": {9114, 9719, 9773, 9774, 9775, 9776},

    "Autres": {
        6011, 6012, 6013, 6014, 6110, 6111, 6112, 6113, 6114, 6115, 6116, 6117, 6118, 6119,
        6120, 6121, 6122, 6123, 6124, 6125, 6126, 6127, 6128, 6129, 6131, 6132, 6133, 6134,
        6135, 6191, 6208, 6209, 6210, 6211, 6212, 6213, 6214, 6215, 6221, 6222, 6231, 6232,
        6233, 6234, 6235, 6236, 6237, 6238, 6239, 6241, 6242, 6251, 6252, 6253, 6254, 6255,
        6256, 6257, 6258, 6261, 6262, 6263, 6264, 6311, 6312, 7111, 7112, 7113, 7119, 8111,
        8112, 8113, 8114, 8115, 8116, 8117, 8118, 8119, 8120, 8121, 8221, 8222, 8223, 8224,
        8225, 8226, 8227, 9111, 9112, 9113, 9115, 9116, 9118, 9119, 9121, 9122, 9123, 9129,
        9131, 9132, 9133, 9134, 9135, 9141, 9142, 9143, 9144, 9151, 9152, 9161, 9162, 9163,
        9164, 9165, 9166, 9167, 9168, 9169, 9170, 9191, 9201, 9202, 9203, 9204, 9205, 9206,
        9207, 9211, 9221, 9311, 9312, 9313, 9318, 9319, 9411, 9412, 9413, 9414, 9415, 9421,
        9422, 9423, 9424, 9425, 9426, 9427, 9429, 9430, 9431, 9432, 9433, 9434, 9435, 9436,
        9511, 9512, 9513, 9514, 9521, 9566, 9567, 9568, 9569, 9570, 9605, 9711, 9712, 9713,
        9714, 9720, 9721, 9722, 9723, 9724, 9725, 9728, 9729, 9730, 9731, 9732, 9733, 9734,
        9735, 9736, 9737, 9741, 9743, 9744, 9751, 9752, 9763, 9764, 9765, 9766, 9767, 9768,
        9769, 9772, 9778
    }
}

dict_IA = {}

for d in [dict_IA1, dict_IA2, dict_IA3, dict_IA4, dict_IA5]:
    for key, value in d.items():
        dict_IA.setdefault(key, set()).update(value)

dict_IA_inversed = {id: id_ for id_, prs_list in dict_IA.items() for id in prs_list}
df_poste_IA = df_poste.with_columns(pl.col(*keys_to_group_by).replace_strict(dict_IA_inversed, default='Autres'))
df_poste_IA = df_poste_IA[all_keys].group_by(keys_to_group_by).agg(pl.col(*all_indicators).sum())
df_poste_IA.write_csv(f'IA.csv', separator=';')
print(df_poste_IA)

shape: (7, 4)
┌─────────────────┬─────────────┬─────────────┬────────────┐
│ PRS_NAT         ┆ FLT_REM_MNT ┆ FLT_PAI_MNT ┆ RAC        │
│ ---             ┆ ---         ┆ ---         ┆ ---        │
│ str             ┆ f64         ┆ f64         ┆ f64        │
╞═════════════════╪═════════════╪═════════════╪════════════╡
│ Pharmacie       ┆ 3.3865e10   ┆ 3.8708e10   ┆ 4.8429e9   │
│ Hospitalisation ┆ 3.4105e10   ┆ 4.2702e10   ┆ 8.5967e9   │
│ Autres          ┆ 8.2723e10   ┆ 6.0523e10   ┆ -2.2200e10 │
│ Optique         ┆ 1.1616e8    ┆ 7.4580e9    ┆ 7.3418e9   │
│ Dentaire        ┆ 3.8740e9    ┆ 1.3402e10   ┆ 9.5283e9   │
│ Auditif         ┆ 4.1522e8    ┆ 1.9599e9    ┆ 1.5447e9   │
│ Spécialistes    ┆ 2.2115e9    ┆ 3.0326e9    ┆ 8.2111e8   │
└─────────────────┴─────────────┴─────────────┴────────────┘
